In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Create image containing a white square with blue background
def create_image(size=256):
    img = np.zeros((size, size, 3), dtype=np.uint8)
    img[:, :, 2] = 255
    start = int(size/4)
    end = int(size*3/4)
    img[start:end, start:end] = 255
    return img

# Create random transformation matrix
def create_random_transformation_matrix(angle_range=(-np.pi*0, np.pi*0), tx_range=(0,0), ty_range=(0,0), sx_range=(1,1), sy_range=(1,1)):
    # Random rotation angle
    angle = np.random.uniform(*angle_range)
    # Random translation
    tx = np.random.uniform(*tx_range)
    ty = np.random.uniform(*ty_range)
    # Random scaling
    sx = np.random.uniform(*sx_range)
    sy = np.random.uniform(*sy_range)
    # Create transformation matrix
    M = np.array([[sx*np.cos(angle), -sy*np.sin(angle), tx],
                  [sx*np.sin(angle),  sy*np.cos(angle), ty],
                  [0, 0, 1]])
    return M

# Apply random transformation to image
def apply_random_transformation(img, M):
    # Apply transformation to image, with origin at center
    # Create translation matrix to move origin to center
    T = np.array([[1, 0, img.shape[1]/2],
                  [0, 1, img.shape[0]/2],
                  [0, 0, 1]])
    # Create inverse translation matrix
    T_inv = np.array([[1, 0, -img.shape[1]/2],
                      [0, 1, -img.shape[0]/2],
                      [0, 0, 1]])
    # Apply transformation
    # M = T_inv.dot(M).dot(T)
    img = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]))
    
    return img

In [ ]:
def get_warped_corners(img, homography_matrix):
    # Get the dimensions of the image
    height, width = img.shape[:2]

    # Get the corners of the original image
    corners = np.array([[0, 0], [0, height], [width, height], [width, 0]], dtype=np.float32)

    # Transform the corners of the original image to the warped image
    warped_corners = cv2.perspectiveTransform(corners.reshape(-1, 1, 2), homography_matrix).reshape(-1, 2)

    return warped_corners


def covers_bounding_box(img, homography_matrix, bounding_box_size):
    # Get the dimensions of the image
    height, width = img.shape[:2]
    
    # Check if the bounding box is still covered by the warped image
    # Get the corners of the warped image
    warped_corners = get_warped_corners(img, homography_matrix)

    # Get the corners of the bounding box
    central_w, central_h = bounding_box_size
    central_x = (width - central_w) // 2
    central_y = (height - central_h) // 2
    bb_corners = [(central_x, central_y), (central_x, central_y + central_h),
                (central_x + central_w, central_y + central_h), (central_x + central_w, central_y)]
    
    # Get the edges of the bounding box and warped image
    bb_edges = [(bb_corners[i], bb_corners[(i + 1) % 4]) for i in range(4)]
    wc_edges = [(warped_corners[i], warped_corners[(i + 1) % 4]) for i in range(4)]

    # Check if any of the bounding box edges intersect with any of the warped image edges
    if all(not intersect(e1, e2) for e1 in bb_edges for e2 in wc_edges):
        # Check if the center of the bounding box is inside the warped image
        return cv2.pointPolygonTest(warped_corners, (central_x + central_w // 2, central_y + central_h // 2), False) >= 0
    return False

def intersect(l1, l2):
    x1, y1 = l1[0]
    x2, y2 = l1[1]
    x3, y3 = l2[0]
    x4, y4 = l2[1]

    # Check if the lines are parallel
    # den is the denominator of ua and ub
    den = (y4 - y3) * (x2 - x1) - (x4 - x3) * (y2 - y1)
    if den == 0:
        return False
    
    # Check if the lines intersect
    ua = ((x4 - x3) * (y1 - y3) - (y4 - y3) * (x1 - x3)) / den
    ub = ((x2 - x1) * (y1 - y3) - (y2 - y1) * (x1 - x3)) / den

    # Check if the intersection is within the line segments
    return 0 <= ua <= 1 and 0 <= ub <= 1


# Test intersection function
l1 = ((0, 0), (1, 1))
l2 = ((0, 1), (1, 0))
print(intersect(l1, l2))
# Plot test lines
# plt.plot([l1[0][0], l1[1][0]], [l1[0][1], l1[1][1]], 'r')
# plt.plot([l2[0][0], l2[1][0]], [l2[0][1], l2[1][1]], 'b')
# plt.show()

In [ ]:
# Create image
img = create_image()
M = create_random_transformation_matrix(
    angle_range=(-np.pi*0.5, np.pi*0.5),
    # tx_range=(-0.1, 0.1),
    # ty_range=(-0.1, 0.1),
    # sx_range=(0.9, 1.1),
    # sy_range=(0.9, 1.1),
)
img = apply_random_transformation(img, M)

# Set colormap to yellow and blue if image doesn't cover bounding box
cmap = 'gray'
bounding_box_size = (img.shape[1]//2, img.shape[0]//2)
if not covers_bounding_box(img, M, bounding_box_size):
    cmap = 'YlGnBu'
    print('Image does not cover bounding box')

# Add warped corners to image in red
warped_corners = get_warped_corners(img, M)
for corner in warped_corners:
    cv2.circle(img, tuple(corner.astype(int)), 5, (255, 0, 0), -1)

# Add bounding box to image in green
central_w, central_h = bounding_box_size
central_x = (img.shape[1] - central_w) // 2
central_y = (img.shape[0] - central_h) // 2
cv2.rectangle(img, (central_x, central_y), (central_x + central_w, central_y + central_h), (0, 255, 0), 2)

# Add wc_lines to image in purple
wc_lines = [(warped_corners[i], warped_corners[(i + 1) % 4]) for i in range(4)]
for line in wc_lines:
    cv2.line(img, tuple(line[0].astype(int)), tuple(line[1].astype(int)), (255, 0, 255), 2)

# Plot image
plt.imshow(img, cmap=cmap)